<a href="https://colab.research.google.com/github/dlwjddn29/trailing-algo-strat1/blob/main/algo1_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1st Commit on GitHub

Quant Strategy Rules: S&P 500 Mid-Cap Trailing Stop Strategy
==============================================================

Universe Selection
------------------
1. Select companies ranked 100th–119th by market capitalization in the S&P 500.
2. Re-rank and update the universe monthly.
3. Optional filters:
   - Exclude high-volatility stocks (prefer lower 6-month volatility).
   - Optionally, prioritize positive 6-month momentum or stable earnings.

Portfolio Allocation
--------------------
4. Allocate capital equally among selected stocks.
   - Example: $100,000 ÷ 20 = $5,000 per stock.
5. Cash from stopped-out positions remains idle (no re-entry until next rebalance).

Entry Rules
------------
6. Buy each selected stock at market open on the rebalance day.
7. (Optional) Adjust position sizes by volatility — smaller for high-vol stocks.

Stop-Loss & Trailing Rules
--------------------------
8. Initial stop-loss: 7% below entry price.
9. Dynamic trailing stop:
   - When price ≥ +3% from entry → move stop to 4% below current price.
   - For each additional +1% gain → recalc stop = 4% below current price.
   - Alternatively, use volatility-adjusted trailing stop (e.g., 2×ATR below price).
10. Update stop-loss levels daily (or intraday if using real-time data).

Exit Rules
----------
11. If price hits stop-loss → sell immediately and mark position as exited.
12. Do not re-enter exited stocks until next monthly rebalance.
13. Open positions remain active (Good-til-cancelled) until stop triggers or rebalance.

Risk Management (Optional Enhancements)
---------------------------------------
14. Cap max portfolio drawdown (e.g., 10–15%) — move all to cash if breached.
15. Limit max sector exposure (e.g., ≤25% in any sector).
16. Track slippage and transaction costs for realistic backtesting.

Rebalancing
------------
17. Rebalance monthly to refresh the 100–119 universe and reallocate idle cash.
18. Update all open positions’ stop-loss levels and portfolio weights at rebalance.


'''

'\n1st Commit on GitHub\n\nQuant Strategy Rules: S&P 500 Mid-Cap Trailing Stop Strategy\n==============================================================\n\nUniverse Selection\n------------------\n1. Select companies ranked 100th–119th by market capitalization in the S&P 500.\n2. Re-rank and update the universe monthly.\n3. Optional filters:\n   - Exclude high-volatility stocks (prefer lower 6-month volatility).\n   - Optionally, prioritize positive 6-month momentum or stable earnings.\n\nPortfolio Allocation\n--------------------\n4. Allocate capital equally among selected stocks.\n   - Example: $100,000 ÷ 20 = $5,000 per stock.\n5. Cash from stopped-out positions remains idle (no re-entry until next rebalance).\n\nEntry Rules\n------------\n6. Buy each selected stock at market open on the rebalance day.\n7. (Optional) Adjust position sizes by volatility — smaller for high-vol stocks.\n\nStop-Loss & Trailing Rules\n--------------------------\n8. Initial stop-loss: 7% below entry price.\

### Import relevant libraries/packages

1.   Stock Ticker Scraper
2.   Generate Market Data
3.   Math Functions

In [ ]:
import requests
import pandas as pd

import yfinance as yf
import logging
logging.getLogger("yfinance").disabled = True

from bs4 import BeautifulSoup
from io import StringIO



In [ ]:
def ticker_data(url):
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        raise ValueError("No HTML tables found on the page. Page may use JavaScript rendering.")

    tables_df = [pd.read_html(StringIO(str(table)))[0] for table in tables]
    print(tables_df)

    #print(len(tables_df)) # - > 1 because the page only has 1 table

    df = None
    for t in tables_df:
    # for loop isn't necessary if page only has 1 table, but filters out only the needed-table, in this case the one with 'Symbol'
        if "Symbol" in t.columns:
            df = t
            break

    if df is None:
        raise ValueError("No table found with a 'Symbol' column.")

    # Clean up symbols (remove whitespace, duplicates)
    df["Symbol"] = df["Symbol"].astype(str).str.strip().str.replace(".", "-", regex=False)
    df = df.drop_duplicates(subset=["Symbol"]).reset_index(drop=True)

    print(f"Found {len(df)} symbols from NerdWallet.")
    return df

url = "https://www.nerdwallet.com/article/investing/sp-500-companies"
symbols = ticker_data(url)["Symbol"].tolist()
print(symbols)





[                          Company Symbol
0                  Microsoft Corp   MSFT
1                     Nvidia Corp   NVDA
2                      Apple Inc.   AAPL
3                  Amazon.com Inc   AMZN
4    Meta Platforms, Inc. Class A   META
..                            ...    ...
499      News Corporation Class B    NWS
500        Amentum Holdings, Inc.   AMTM
501                AppLovin Corp.    APP
502             Robinhood Markets   HOOD
503              Emcor Group Inc.    EME

[504 rows x 2 columns]]
Found 504 symbols from NerdWallet.
['MSFT', 'NVDA', 'AAPL', 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA', 'BRK-B', 'GOOG', 'JPM', 'V', 'LLY', 'NFLX', 'MA', 'COST', 'XOM', 'WMT', 'PG', 'JNJ', 'HD', 'ABBV', 'BAC', 'UNH', 'KO', 'PM', 'CRM', 'ORCL', 'CSCO', 'GE', 'PLTR', 'IBM', 'WFC', 'ABT', 'MCD', 'CVX', 'LIN', 'NOW', 'DIS', 'ACN', 'T', 'ISRG', 'MRK', 'UBER', 'GS', 'INTU', 'VZ', 'AMD', 'ADBE', 'RTX', 'PEP', 'BKNG', 'TXN', 'QCOM', 'PGR', 'CAT', 'SPGI', 'AXP', 'MS', 'BSX', 'BA', 'TMO', '

In [ ]:
tickers = yf.Tickers(symbols)

info_list = []
for sym, ticker in tickers.tickers.items():
    try:
        info = ticker.info
        market_cap = info.get("marketCap", None)
        if market_cap is None:
            continue
        info_list.append({
            "Symbol": sym,
            "MarketCap": market_cap,
            "Sector": info.get("sector", None),
            "CompanyName": info.get("shortName", None)
        })
    except Exception as e:
        print(f"⚠️ Skipping {sym}: {e}")
        continue

df_info = pd.DataFrame(info_list)
#df_info = df_info.dropna(subset=["MarketCap"]).sort_values("MarketCap", ascending=False)

In [ ]:
valid_symbols = df_info["Symbol"].tolist()
skipped = [s for s in symbols if s not in valid_symbols]
print(f"Skipped {len(skipped)} tickers:", skipped)

Skipped 5 tickers: ['HES', 'ANSS', 'JNPR', 'WBA', 'PARA']


In [ ]:
print(df_info)

    Symbol      MarketCap                  Sector              CompanyName
0     MSFT  3692554092544              Technology    Microsoft Corporation
1     NVDA  4580888281088              Technology       NVIDIA Corporation
2     AAPL  3967007326208              Technology               Apple Inc.
3     AMZN  2612795801600       Consumer Cyclical         Amazon.com, Inc.
4     META  1567037325312  Communication Services     Meta Platforms, Inc.
..     ...            ...                     ...                      ...
494    NWS    15625144320  Communication Services         News Corporation
495   AMTM     5452994560             Industrials   Amentum Holdings, Inc.
496    APP   209508450304  Communication Services     Applovin Corporation
497   HOOD   117218263040      Financial Services  Robinhood Markets, Inc.
498    EME    29068267520             Industrials        EMCOR Group, Inc.

[499 rows x 4 columns]
